# Deception example

In [1]:
%matplotlib notebook

In [2]:
import numpy as np
import networkx as nx
from copy import deepcopy

import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.patches import Circle

from stonesoup.custom.graph import load_graph_dict, get_xy_from_range_edge, CustomDiGraph
from stonesoup.custom.plotting import plot_network, highlight_edges, highlight_nodes,     plot_cov_ellipse
from stonesoup.custom.simulation import simulate
from stonesoup.initiator.destination import DestinationBasedInitiator
from stonesoup.models.transition.destination import DestinationTransitionModel
from stonesoup.models.measurement.destination import DestinationMeasurementModel
from stonesoup.predictor.particle import ParticlePredictor2
from stonesoup.updater.particle import ParticleUpdater2
from stonesoup.resampler.particle import SystematicResampler2
from stonesoup.types.hypothesis import SingleHypothesis
from stonesoup.types.detection import Detection

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
seed = 900 #163321 #900
np.random.seed(seed)

# Global vars
PLOT = True
num_particles = 1000
source = 402 #519
destination = 115
speed = 0.1
zoom = 0.2
path = r'C:\Users\sglvladi\OneDrive\Workspace\PostDoc\CADMURI\MATLAB\simple\data\minn_2.mat'
S = load_graph_dict(path)
S2 = deepcopy(S)
for i in range(len(S2['Edges']['Weight'])):
    S2['Edges']['Weight'][i] += np.random.normal(0, 200)
    if S2['Edges']['Weight'][i] <= 0:
        S2['Edges']['Weight'][i] = 0.01
G = CustomDiGraph.from_dict(S2)
G2 = CustomDiGraph.from_dict(S)
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()
num_destinations = 100 # num_nodes

C:\Users\sglvladi\Anaconda3\envs\bsp\lib\site-packages\scipy\io\matlab\_mio.py:226: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


In [4]:
def prepare_plot(G, short_paths_e, gnd_route_e, destination, destinations):
    fig = plt.figure(figsize=(15, 7))
    gs = fig.add_gridspec(2, 3)
    ax1 = fig.add_subplot(gs[:, :2])
    ax2 = fig.add_subplot(gs[0, 2])
    ax3 = fig.add_subplot(gs[1, 2])
    plot_network(G, ax1)
    plot_network(G, ax2)
    for key, value in short_paths_e.items():
        highlight_edges(G, ax1, value, edge_color='y')
        highlight_edges(G, ax2, value, edge_color='y')
    highlight_edges(G, ax1, gnd_route_e, edge_color='g')
    highlight_edges(G, ax2, gnd_route_e, edge_color='g')
    highlight_nodes(G, ax1, destinations, node_color='m', node_size=10)
    highlight_nodes(G, ax2, destinations, node_color='m', node_size=10)
    highlight_nodes(G, ax1, [destination], node_color='r', node_size=10)
    highlight_nodes(G, ax2, [destination], node_color='r', node_size=10)
    ax1.plot([], [], '-g', label='True path')
    ax1.plot([], [], 'sr', label='True destination')
    ax1.plot([], [], '-y', label='Confuser paths')
    ax1.plot([], [], 'sm', label='Confuser destinations')
    ax1.legend(loc='upper right')
    ax1.set_title('Global view')
    ax2.plot([], [], 'r.', label='Particles')
    ax2.plot([], [], 'b-', label='Trajectory')
    ax2.plot([], [], 'cx', label='Measurements')
    ax2.set_title('Zoomed view')
    ax2.legend(loc='upper right')
    return dict(fig=fig, axes=[ax1, ax2, ax3], arts=[[],[],[]])

In [5]:
def plot(cfg, track1, track2, detections, G1, G2, ratio, idx):
    arts1, arts2, arts3 = cfg['arts']
    ax1, ax2, ax3 = cfg['axes']
    arts1.clear()
    arts2.clear()
    arts3.clear()

    detection_data = np.array([detection.state_vector for detection in detections])
    arts1.append(ax1.plot(detection_data[:, 0], detection_data[:, 1], 'xc', label="Detections")[0])
    arts2.append(ax2.plot(detection_data[:, 0], detection_data[:, 1], 'xc', label="Detections")[0])

    plot_track(cfg, track1, G1, idx)
    plot_track(cfg, track2, G2, idx)

    # ax3.cla()
    arts3.append(ax3.plot(ratio[:idx+1], 'g-')[0])
    ax3.set_title(r"Probability of deceptive behaviour ($P_{D}$) vs Time")
    ax3.set_ylabel(r'$P_D$')
    ax3.set_xlabel('Time (s)')


In [6]:
def plot_track(cfg, track, G, idx):
    posterior = track.states[idx]

    # Compute statistics
    data = posterior.particles
    pos = nx.get_node_attributes(G, 'pos')
    est_dest_pos = np.array([list(pos[node]) for node in data[3, :]]).T + np.atleast_2d(np.random.multivariate_normal([0, 0], np.diag([0.0001, 0.0001]), num_particles)).T


    # Plot
    # plot_network(G, ax)
    arts1, arts2, arts3 = cfg['arts']
    ax1, ax2, ax3 = cfg['axes']

    xy = get_xy_from_range_edge(data[0, :], data[2, :], G)
    arts1.append(ax1.plot(xy[0, :], xy[1, :], '.r')[0])
    arts2.append(ax2.plot(xy[0, :], xy[1, :], '.r')[0])
    try:
        mu = np.average(est_dest_pos, axis=1, weights=posterior.weights)
        cov = np.cov(est_dest_pos, ddof=0, aweights=posterior.weights)
    except ZeroDivisionError:
        weights = np.ones((len(posterior),)) / len(posterior)
        mu = np.average(est_dest_pos, axis=1, weights=weights)
        cov = np.cov(est_dest_pos, ddof=0, aweights=weights)

    if np.trace(cov) > 1e-2:
        arts1.append(plot_cov_ellipse(cov, mu, ax=ax1, nstd=3, fill=None, edgecolor='r'))
    else:
        circ = Circle(mu, 0.2, fill=None, edgecolor='r')
        ax1.add_artist(circ)
        arts1.append(circ)

    mu = np.mean(xy, axis=1)
    arts1.append(ax1.plot([mu[0] - zoom, mu[0] + zoom, mu[0] + zoom, mu[0] - zoom, mu[0] - zoom],
                          [mu[1] - zoom, mu[1] - zoom, mu[1] + zoom, mu[1] + zoom, mu[1] - zoom],
                          '-k')[0])
    ax2.set_xlim((mu[0] - zoom, mu[0] + zoom))
    ax2.set_ylim((mu[1] - zoom, mu[1] + zoom))

    return cfg

Simulate ground-truth

In [7]:
gnd_path, gnd_route_n, gnd_route_e = simulate(G, source, destination, speed)

Pre-compute short_paths

In [8]:
feed = [destination]
feed_tmp = set([i for i in range(num_nodes)])-set(feed)
destinations = feed + list(np.random.choice(list(feed_tmp),(num_destinations-len(feed),), False)) # [i for i in range(num_nodes)]
targets = None if len(destinations) == num_nodes else destinations
short_paths_n, short_paths_e = G2.shortest_path(source, targets)
a = G.shortest_path(source, targets)

Simulate detections

In [9]:
# Measurement model
mapping = [0, 1]
R = np.eye(2) * 0.0002
measurement_model = DestinationMeasurementModel(ndim_state=4, mapping=mapping, noise_covar=R,
                                                graph=G)

# Simulate detections
scans = []
for gnd_state in gnd_path:
    gnd_sv = gnd_state.state_vector
    det_sv = gnd_sv + measurement_model.rvs()
    timestamp = gnd_state.timestamp
    metadata = {"gnd_id": gnd_path.id}
    detection = Detection(state_vector=det_sv, timestamp=timestamp, metadata=metadata)
    scans.append((timestamp, {detection}))

## Filter 1

In [10]:
# Transition model
transition_model1 = DestinationTransitionModel(0.001, G)

# Measurement model
mapping = [0, 1]
R = np.eye(2) * 0.0002
measurement_model1 = DestinationMeasurementModel(ndim_state=4, mapping=mapping, noise_covar=R,
                                                 graph=G)

# Initiator track
initiator1 = DestinationBasedInitiator(measurement_model1, num_particles, speed, G)

# Predictor
predictor1 = ParticlePredictor2(transition_model1)

# Updater
updater1 = ParticleUpdater2(measurement_model1, SystematicResampler2())

# Initiate track
tracks = initiator1.initiate(scans[0][1])
track1 = next(t for t in tracks)

## Filter 2

In [11]:
# Transition model
transition_model2 = DestinationTransitionModel(0.001, G2)

# Measurement model
mapping = [0, 1]
R = np.eye(2) * 0.0002
measurement_model2 = DestinationMeasurementModel(ndim_state=4, mapping=mapping, noise_covar=R,
                                                 graph=G2)

# Initiator track
initiator2 = DestinationBasedInitiator(measurement_model2, num_particles, speed, G2)

# Predictor
predictor2 = ParticlePredictor2(transition_model2)

# Updater
updater2 = ParticleUpdater2(measurement_model2, SystematicResampler2())

# Initiate track
tracks = initiator2.initiate(scans[0][1])
track2 = next(t for t in tracks)

In [12]:
lik1 = []
lik2 = []
ratio = []
est1 = np.array([[], []])
est2 = np.array([[], []])
arts = []
for timestamp, detections in scans:
#     print(timestamp)
    detection = list(detections)[0]

    # Run PF1
    prediction1 = predictor1.predict(track1.state, timestamp=timestamp)
    hypothesis1 = SingleHypothesis(prediction1, detection)
    posterior1 = updater1.update(hypothesis1)
    track1.append(posterior1)

    # Run PF2
    prediction2 = predictor2.predict(track2.state, timestamp=timestamp)
    hypothesis2 = SingleHypothesis(prediction2, detection)
    posterior2 = updater2.update(hypothesis2)
    track2.append(posterior2)

    lik1.append(np.sum(track1.weights))
    lik2.append(np.sum(track2.weights))
    ratio.append(lik1[-1]/(lik1[-1]+lik2[-1]))

In [13]:
if PLOT:
    cfg = prepare_plot(G, short_paths_e, gnd_route_e, destination, destinations)

for i in range(len(scans)):
    if PLOT:
        plot(cfg, track1, track2, scans[i][1], G, G2, ratio, i)
        arts1, arts2, arts3 = cfg['arts']
        arts.append([*arts1, *arts2, *arts3])

ani2 = animation.ArtistAnimation(cfg['fig'], arts, interval=20, blit=True, repeat_delay=200)

<IPython.core.display.Javascript object>

C:\Users\sglvladi\Anaconda3\envs\bsp\lib\site-packages\numpy\lib\function_base.py:2680: RuntimeWarning: overflow encountered in true_divide
  c *= np.true_divide(1, fact)
